In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from calibration_utils import calibrate_camera, undistort
from binarization_utils import binarize


yellow_HSV_th_min = np.array([0, 70, 70])
yellow_HSV_th_max = np.array([50, 255, 255])


def thresh_frame_in_HSV(frame, min_values, max_values, verbose=False):
    
    HSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    min_th_ok = np.all(HSV > min_values, axis=2)
    max_th_ok = np.all(HSV < max_values, axis=2)

    out = np.logical_and(min_th_ok, max_th_ok)

    return out


def get_binary_from_equalized_grayscale(frame):
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    eq_global = cv2.equalizeHist(gray)

    _, th = cv2.threshold(eq_global, thresh=250, maxval=255, type=cv2.THRESH_BINARY)

    return th

def birdeye(img, verbose=False):

    h, w = img.shape[:2]

    src = np.float32([[w, h-10],    
                      [0, h-10],    
                      [555, 460],   
                      [745, 460]])  
    dst = np.float32([[w, h],       
                      [0, h],       
                      [0, 0],       
                      [w, 0]])      

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)

    warped = cv2.warpPerspective(img, M, (w, h), flags=cv2.INTER_LINEAR)

    return warped, M, Minv

if __name__ == '__main__':

    ret, mtx, dist, rvecs, tvecs = calibrate_camera(calib_images_dir='camera_cal')

    for test_img in glob.glob('test_images/*.jpg'):

        img = cv2.imread(test_img)

        img_undistorted = undistort(img, mtx, dist, verbose=False)
        
        img_birdeye, M, Minv = birdeye(cv2.cvtColor(img_undistorted, cv2.COLOR_BGR2RGB), verbose=True)
        
        h, w = img.shape[:2]

        binary = np.zeros(shape=(h, w), dtype=np.uint8)

        HSV_yellow_mask = thresh_frame_in_HSV(img_birdeye, yellow_HSV_th_min, yellow_HSV_th_max, verbose=False)
        binary = np.logical_or(binary, HSV_yellow_mask)

        eq_white_mask = get_binary_from_equalized_grayscale(img_birdeye)

        cv2.imshow("warped", eq_white_mask)
        cv2.waitKey(0)

Done.


In [69]:
import numpy as np
import sys
import cv2 as cv
def show_wait_destroy(winname, img):
    cv.imshow(winname, img)
    cv.moveWindow(winname, 500, 0)
    cv.waitKey(0)
    cv.destroyWindow(winname)
def main():
    src = cv.imread("xxx.jpg", cv.IMREAD_COLOR)
    src = cv2.resize(src,(416,416))
    
    btu = cv2.imread('btu.jpg')
    btu = cv2.resize(btu,(50,50))
    carf = cv2.imread('carf.jpg')
    carf = cv2.resize(carf,(50,50))
    
    if len(src.shape) != 2:
        gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
    else:
        gray = src
    #show_wait_destroy("gray", gray)
    gray = cv.bitwise_not(gray)
    bw = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, \
                                cv.THRESH_BINARY, 15, -2)
    #show_wait_destroy("binary", bw)
    
    horizontal = np.copy(bw)
    cols = horizontal.shape[1]
    horizontal_size = cols // 30
    horizontalStructure = cv.getStructuringElement(cv.MORPH_RECT, (horizontal_size, 7))
    horizontal = cv.erode(horizontal, horizontalStructure)
    horizontal = cv.dilate(horizontal, horizontalStructure)
    #show_wait_destroy("horizontal", horizontal)
    
    edges = cv.adaptiveThreshold(horizontal, 255, cv.ADAPTIVE_THRESH_MEAN_C,
                                cv.THRESH_BINARY, 3, -2)
    #show_wait_destroy("edges", edges)
    
    kernel = np.ones((2, 2), np.uint8)
    edges = cv.erode(edges, kernel)
    #show_wait_destroy("erode", edges)
    
    smooth = np.copy(horizontal)
    smooth = cv.blur(smooth, (2, 2))
    (rows, cols) = np.where(edges != 0)
    horizontal[rows, cols] = smooth[rows, cols]
    #show_wait_destroy("smooth - final", horizontal)
    
    lines = cv2.HoughLines(horizontal,5,np.pi/180,255)
    for rho,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho 
        y0 = b*rho 
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))

        cv2.line(src,(x1+300,y1),(x2+300,y2),(0,0,255),7)
    
        #show_wait_destroy("result", src)
    
    mask = src.copy()
    mask = cv2.rectangle(mask, pt1=(0, 0), pt2=(416, 60), color=(0, 0, 0), thickness=cv2.FILLED)
    blend_on_road = cv2.addWeighted(src1=mask, alpha=0.2, src2=src, beta=0.8, gamma=0) 
    blend_on_road[0:50, 5:55, :] = btu
    blend_on_road[0:50, 60:110, :] = carf
    
    cv2.imshow("image", blend_on_road)
    cv2.waitKey(0)
    
    return 0
if __name__ == "__main__":
    main()